In [1]:
from create_inputs_utils import * 
import os
import numpy as np
import h5py
import json
from tqdm import tqdm
from collections import Counter
from random import seed, choice, sample
import pickle
import argparse
import glob
import logging
import os
import random
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm, trange
import csv
import logging
logger = logging.getLogger(__name__)

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from transformers import (WEIGHTS_NAME, BertConfig,
                                  BertForSequenceClassification, BertTokenizer,
                                  )

dataset = 'coco'
karpathy_json_path='../data/caption_datasets/dataset_coco.json'
captions_per_image=5
output_folder='../preprocessed_data'
base_filename = 'preprocessed_' + dataset
max_len=50

train_image_captions = []
val_image_captions = []
test_image_captions = []

train_image_det = []
val_image_det = []
test_image_det = []

with open(karpathy_json_path, 'r') as j:
    data = json.load(j)
with open(os.path.join(output_folder,'train36_imgid2idx.pkl'), 'rb') as j:
    train_data = pickle.load(j)       
with open(os.path.join(output_folder,'val36_imgid2idx.pkl'), 'rb') as j:
    val_data = pickle.load(j)
    
processor = CaptionProcessor()
task_name = "sst-2"#임의 설정 (필요없는 값)
output_mode = "classification" #임의 설정 (필요없는 값)
model_name_or_path = "bert-base-uncased" 
max_seq_length = 50
device = torch.device("cuda:0" if torch.cuda.is_available()  else "cpu")
label_list = processor.get_labels()
num_labels = len(label_list)

config_class = BertConfig
model_class = BertForSequenceClassification
tokenizer_class = BertTokenizer

config = config_class.from_pretrained(model_name_or_path, num_labels=num_labels, finetuning_task = task_name)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case = True)
#model = model_class.from_pretrained(model_name_or_path, from_tf=bool('.ckpt' in model_name_or_path), config=config)

#model.to(device)    

I1201 13:38:43.279632 139725318354752 file_utils.py:39] PyTorch version 0.4.1 available.
I1201 13:38:47.842504 139725318354752 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I1201 13:38:47.847089 139725318354752 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_

In [2]:
train_image_captions_with_len = []
val_image_captions_with_len = []
test_image_captions_with_len = []

for img in data['images']:
    captions = []
    captions_with_len = []
    
    for caption in img['sentences']:
        # Update word frequency
        if len(caption['tokens']) <= max_len:
            #captions_forlen : [['a','man','with','a','red','helmet'],['a','man',..]] 5개의 캡션씩 들어 있음
            #['a','man','with','a','red','helmet'] -> "a man with a red helmet"
            
            #bowonko
            caption_len = len(caption['tokens']) + 2
            
            caption_sen = " ".join(caption['tokens'])
             
            captions.append(caption_sen)
            
            #bowonko
            
            cwl_item = (caption_sen,caption_len)
            captions_with_len.append(cwl_item)
            
    #captions : ['sen1','sen2','sen3','sen4','sen5']
    if len(captions) == 0:
        print('절대 발생할 수 없음')
        continue

    #ID
    image_id = img['filename'].split('_')[2]
    image_id = int(image_id.lstrip("0").split('.')[0])

    #split은 train, val, test, restval 로 구성
    if img['split'] in {'train', 'restval'}:
        if img['filepath'] == 'train2014':
            if image_id in train_data:
                train_image_det.append(("t",train_data[image_id]))
        else:
            if image_id in val_data:
                train_image_det.append(("v",val_data[image_id]))
        #bowonko
        train_image_captions.append(captions)
        train_image_captions_with_len.append(captions_with_len)
        
        #train_image_captions은 2차원, element(captions) = ['sen1','sen2','sen3','sen4','sen5'], 

    elif img['split'] in {'val'}:
        if image_id in val_data:
            val_image_det.append(("v",val_data[image_id]))   
        #bowonko    
        val_image_captions.append(captions)
        val_image_captions_with_len.append(captions_with_len)
        
        #val_image_captions은 2차원, element(captions) = ['sen1','sen2','sen3','sen4','sen5']
    elif img['split'] in {'test'}:
        if image_id in val_data:
            test_image_det.append(("v",val_data[image_id])) 
        #bowonko
        test_image_captions.append(captions)
        test_image_captions_with_len.append(captions_with_len)
        
        #test_image_captions은 2차원, element(captions) = ['sen1','sen2','sen3','sen4','sen5']
# Sanity check
assert len(train_image_det) == len(train_image_captions)
assert len(train_image_det) == len(train_image_captions_with_len)
assert len(val_image_det) == len(val_image_captions)
assert len(val_image_det) == len(val_image_captions_with_len)
assert len(test_image_det) == len(test_image_captions)
assert len(test_image_det) == len(test_image_captions_with_len)

caption_size = 5

#captions_forlen : [['a','man','with','a','red','helmet'],['a','man',..]]
for impaths, imcaps, split in [(train_image_det, train_image_captions_with_len, 'TRAIN'),
                                   (val_image_det, val_image_captions_with_len, 'VAL'),
                                   (test_image_det, test_image_captions_with_len, 'TEST')]:
    
    #imcaps == image_captions_with_len > captions_with_len > (caption_sen,caption_len)
    #cwl_item = (caption_sen,caption_len)
    #captions_with_len.append(cwl_item)
    
    caplens = []
    image_captions = []
    enc_captions = []
    
    #captions에 (caption_sen,caption_len) 다 들어가게 변경했으므로 captions_sen에 caption_sen만 모아놓음
    captions_sen = []
    
    for i, path in enumerate(tqdm(impaths)):
        image_captions = []
        
        # Sample captions
        #이런 일은 발생하지 않음
        if len(imcaps[i]) < captions_per_image:#['sen1','sen2','sen3','sen4','sen5']
            #이미지 당 caption이 5개보다 적으면 중복해서 구성
            captions = imcaps[i] + [choice(imcaps[i]) for _ in range(captions_per_image - len(imcaps[i]))]
        else: #모두 여기로 들어옴
            captions = sample(imcaps[i], k=captions_per_image)

        # Sanity check
        assert len(captions) == captions_per_image
                
        for cap_sen,cap_len in captions:
            caplens.append(cap_len)
            captions_sen.append(cap_sen)
        
        image_captions.append(captions_sen)
        
    
    
    with open(os.path.join(output_folder, split + '_CAPLENS_' + base_filename + '.json'), 'w') as j:
        json.dump(caplens, j)
    
    data_dir = output_folder


    #목적 load_and_cache_example을 불러서 cache 파일 만들기 이후 두 번째 실행할 때는 load 
    if(split == "TRAIN"): 
        #TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        train_dataset = load_and_cache_examples(processor,tokenizer,max_seq_length,model_name_or_path,image_captions,data_dir,split)
        caplens_train = []
        batch_size = len(train_dataset)
        
        for b in range(batch_size):
            train_caption = train_dataset[b][0]
            cnt = 0
            for i in range(len(train_caption)):
                if(train_caption[i] == 0):
                    break
                else:
                    cnt += 1
            caplens_train.append(cnt)        
                
          
        with open(os.path.join(output_folder, 'TRAIN' + '_CAPLENS_' + base_filename + '.json'), 'w') as j:
            json.dump(caplens_train, j)
        
        
    elif(split == "VAL"):
        dev_dataset = load_and_cache_examples(processor,tokenizer,max_seq_length,model_name_or_path,image_captions,data_dir,split)
        caplens_dev = []
        batch_size = len(dev_dataset)
        
        for b in range(batch_size):
            cnt = 0
            dev_caption = dev_dataset[b][0]
            for i in range(len(dev_caption)):
                if(dev_caption[i] == 0):
                    break
                else:
                    cnt += 1
            caplens_dev.append(cnt)
                                  
        with open(os.path.join(output_folder, 'VAL' + '_CAPLENS_' + base_filename + '.json'), 'w') as j:
            json.dump(caplens_dev, j)
        
    elif(split == "TEST"):
        test_dataset = load_and_cache_examples(processor,tokenizer,max_seq_length,model_name_or_path,image_captions,data_dir,split)
        caplens_test = []
        batch_size = len(test_dataset)
        for b in range(batch_size):
            cnt = 0
            test_caption = test_dataset[b][0]
            for i in range(len(test_caption)):
                if(test_caption[i] == 0):
                    break
                else:
                    cnt += 1
            caplens_test.append(cnt)
                
                    
        with open(os.path.join(output_folder, 'TEST' + '_CAPLENS_' + base_filename + '.json'), 'w') as j:
            json.dump(caplens_test, j)    

            
    
# Save bottom up features indexing to JSON files
with open(os.path.join(output_folder, 'TRAIN' + '_GENOME_DETS_' + base_filename + '.json'), 'w') as j:
    json.dump(train_image_det, j)

with open(os.path.join(output_folder, 'VAL' + '_GENOME_DETS_' + base_filename + '.json'), 'w') as j:
    json.dump(val_image_det, j)

with open(os.path.join(output_folder, 'TEST' + '_GENOME_DETS_' + base_filename + '.json'), 'w') as j:
    json.dump(test_image_det, j)
   


100%|██████████| 113287/113287 [00:01<00:00, 103118.00it/s]
I1201 13:38:52.146618 139725318354752 create_inputs_utils.py:128] Writing example 0 of 566435
I1201 13:38:52.147467 139725318354752 create_inputs_utils.py:218] *** Example ***
I1201 13:38:52.147825 139725318354752 create_inputs_utils.py:219] guid: TRAIN-0-0
I1201 13:38:52.148168 139725318354752 create_inputs_utils.py:221] tokens: [CLS] a woman marking a cake with the back of a chefs knife [SEP]
I1201 13:38:52.148506 139725318354752 create_inputs_utils.py:222] input_ids: 101 1037 2450 10060 1037 9850 2007 1996 2067 1997 1037 27828 5442 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
I1201 13:38:52.148839 139725318354752 create_inputs_utils.py:223] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
I1201 13:38:52.149182 139725318354752 create_inputs_utils.py:224] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 